Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Crawl data

In [ ]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time
import re
from google.colab import files

# Step 1: Define the list of country codes and names
country_codes = [
    ("AF", "Afghanistan"),
    ("AX", "Åland Islands"),
    ("AL", "Albania"),
    ("DZ", "Algeria"),
    ("AS", "American Samoa"),
    ("AD", "Andorra"),
    ("AO", "Angola"),
    ("AI", "Anguilla"),
    ("AQ", "Antarctica"),
    ("AG", "Antigua and Barbuda"),
    ("AR", "Argentina"),
    ("AM", "Armenia"),
    ("AW", "Aruba"),
    ("AU", "Australia"),
    ("AT", "Austria"),
    ("AZ", "Azerbaijan"),
    ("BS", "Bahamas"),
    ("BH", "Bahrain"),
    ("BD", "Bangladesh"),
    ("BB", "Barbados"),
    ("BY", "Belarus"),
    ("BE", "Belgium"),
    ("BZ", "Belize"),
    ("BJ", "Benin"),
    ("BM", "Bermuda"),
    ("BT", "Bhutan"),
    ("BO", "Bolivia"),
    ("BQ", "Bonaire, Sint Eustatius and Saba"),
    ("BA", "Bosnia and Herzegovina"),
    ("BW", "Botswana"),
    ("BR", "Brazil"),
    ("IO", "British Indian Ocean Territory"),
    ("BN", "Brunei Darussalam"),
    ("BG", "Bulgaria"),
    ("BF", "Burkina Faso"),
    ("BI", "Burundi"),
    ("KH", "Cambodia"),
    ("CM", "Cameroon"),
    ("CA", "Canada"),
    ("CV", "Cape Verde"),
    ("KY", "Cayman Islands"),
    ("CF", "Central African Republic"),
    ("TD", "Chad"),
    ("CL", "Chile"),
    ("CN", "China"),
    ("CX", "Christmas Island"),
    ("CC", "Cocos (Keeling) Islands"),
    ("CO", "Colombia"),
    ("KM", "Comoros"),
    ("CG", "Congo"),
    ("CD", "Congo, The Democratic Republic of the"),
    ("CK", "Cook Islands"),
    ("CR", "Costa Rica"),
    ("CI", "Côte d'Ivoire"),
    ("HR", "Croatia"),
    ("CU", "Cuba"),
    ("CW", "Curaçao"),
    ("CY", "Cyprus"),
    ("CZ", "Czech Republic"),
    ("DK", "Denmark"),
    ("DJ", "Djibouti"),
    ("DM", "Dominica"),
    ("DO", "Dominican Republic"),
    ("EC", "Ecuador"),
    ("EG", "Egypt"),
    ("SV", "El Salvador"),
    ("GQ", "Equatorial Guinea"),
    ("ER", "Eritrea"),
    ("EE", "Estonia"),
    ("SZ", "Eswatini"),
    ("ET", "Ethiopia"),
    ("FK", "Falkland Islands (Malvinas)"),
    ("FO", "Faroe Islands"),
    ("FJ", "Fiji"),
    ("FI", "Finland"),
    ("FR", "France"),
    ("GF", "French Guiana"),
    ("PF", "French Polynesia"),
    ("TF", "French Southern Territories"),
    ("GA", "Gabon"),
    ("GM", "Gambia"),
    ("GE", "Georgia"),
    ("DE", "Germany"),
    ("GH", "Ghana"),
    ("GI", "Gibraltar"),
    ("GR", "Greece"),
    ("GL", "Greenland"),
    ("GD", "Grenada"),
    ("GP", "Guadeloupe"),
    ("GU", "Guam"),
    ("GT", "Guatemala"),
    ("GG", "Guernsey"),
    ("GN", "Guinea"),
    ("GW", "Guinea-Bissau"),
    ("GY", "Guyana"),
    ("HT", "Haiti"),
    ("HM", "Heard Island and McDonald Islands"),
    ("VA", "Holy See (Vatican City State)"),
    ("HN", "Honduras"),
    ("HK", "Hong Kong"),
    ("HU", "Hungary"),
    ("IS", "Iceland"),
    ("IN", "India"),
    ("ID", "Indonesia"),
    ("XZ", "Installations in International Waters"),
    ("IR", "Iran, Islamic Republic of"),
    ("IQ", "Iraq"),
    ("IE", "Ireland"),
    ("IM", "Isle of Man"),
    ("IL", "Israel"),
    ("IT", "Italy"),
    ("JM", "Jamaica"),
    ("JP", "Japan"),
    ("JE", "Jersey"),
    ("JO", "Jordan"),
    ("KZ", "Kazakhstan"),
    ("KE", "Kenya"),
    ("KI", "Kiribati"),
    ("KP", "Korea, Democratic People's Republic of"),
    ("KR", "Korea, Republic of"),
    ("KW", "Kuwait"),
    ("KG", "Kyrgyzstan"),
    ("LA", "Lao People's Democratic Republic"),
    ("LV", "Latvia"),
    ("LB", "Lebanon"),
    ("LS", "Lesotho"),
    ("LR", "Liberia"),
    ("LY", "Libya"),
    ("LI", "Liechtenstein"),
    ("LT", "Lithuania"),
    ("LU", "Luxembourg"),
    ("MO", "Macao"),
    ("MG", "Madagascar"),
    ("MW", "Malawi"),
    ("MY", "Malaysia"),
    ("MV", "Maldives"),
    ("ML", "Mali"),
    ("MT", "Malta"),
    ("MH", "Marshall Islands"),
    ("MQ", "Martinique"),
    ("MR", "Mauritania"),
    ("MU", "Mauritius"),
    ("YT", "Mayotte"),
    ("MX", "Mexico"),
    ("FM", "Micronesia, Federated States of"),
    ("MD", "Moldova, Republic of"),
    ("MC", "Monaco"),
    ("MN", "Mongolia"),
    ("ME", "Montenegro"),
    ("MS", "Montserrat"),
    ("MA", "Morocco"),
    ("MZ", "Mozambique"),
    ("MM", "Myanmar"),
    ("NA", "Namibia"),
    ("NR", "Nauru"),
    ("NP", "Nepal"),
    ("NL", "Netherlands"),
    ("NC", "New Caledonia"),
    ("NZ", "New Zealand"),
    ("NI", "Nicaragua"),
    ("NE", "Niger"),
    ("NG", "Nigeria"),
    ("NU", "Niue"),
    ("NF", "Norfolk Island"),
    ("MK", "North Macedonia"),
    ("MP", "Northern Mariana Islands"),
    ("NO", "Norway"),
    ("OM", "Oman"),
    ("PK", "Pakistan"),
    ("PW", "Palau"),
    ("PS", "Palestine, State of"),
    ("PA", "Panama"),
    ("PG", "Papua New Guinea"),
    ("PY", "Paraguay"),
    ("PE", "Peru"),
    ("PH", "Philippines"),
    ("PN", "Pitcairn"),
    ("PL", "Poland"),
    ("PT", "Portugal"),
    ("PR", "Puerto Rico"),
    ("QA", "Qatar"),
    ("RE", "Reunion"),
    ("RO", "Romania"),
    ("RU", "Russian Federation"),
    ("RW", "Rwanda"),
    ("BL", "Saint Barthélemy"),
    ("SH", "Saint Helena, Ascension and Tristan Da Cunha"),
    ("KN", "Saint Kitts and Nevis"),
    ("LC", "Saint Lucia"),
    ("MF", "Saint Martin (French Part)"),
    ("PM", "Saint Pierre and Miquelon"),
    ("VC", "Saint Vincent and the Grenadines"),
    ("WS", "Samoa"),
    ("SM", "San Marino"),
    ("ST", "Sao Tome and Principe"),
    ("SA", "Saudi Arabia"),
    ("SN", "Senegal"),
    ("RS", "Serbia"),
    ("SC", "Seychelles"),
    ("SL", "Sierra Leone"),
    ("SG", "Singapore"),
    ("SX", "Sint Maarten (Dutch Part)"),
    ("SK", "Slovakia"),
    ("SI", "Slovenia"),
    ("SB", "Solomon Islands"),
    ("SO", "Somalia"),
    ("ZA", "South Africa"),
    ("GS", "South Georgia and the South Sandwich Islands"),
    ("SS", "South Sudan"),
    ("ES", "Spain"),
    ("LK", "Sri Lanka"),
    ("SD", "Sudan"),
    ("SR", "Suriname"),
    ("SJ", "Svalbard and Jan Mayen"),
    ("SE", "Sweden"),
    ("CH", "Switzerland"),
    ("SY", "Syrian Arab Republic"),
    ("TW", "Taiwan, Province of China"),
    ("TJ", "Tajikistan"),
    ("TZ", "Tanzania, United Republic of"),
    ("TH", "Thailand"),
    ("TL", "Timor-Leste"),
    ("TG", "Togo"),
    ("TK", "Tokelau"),
    ("TO", "Tonga"),
    ("TT", "Trinidad and Tobago"),
    ("TN", "Tunisia"),
    ("TR", "Türkiye"),
    ("TM", "Turkmenistan"),
    ("TC", "Turks and Caicos Islands"),
    ("TV", "Tuvalu"),
    ("UG", "Uganda"),
    ("UA", "Ukraine"),
    ("AE", "United Arab Emirates"),
    ("GB", "United Kingdom"),
    ("US", "United States"),
    ("UM", "United States Minor Outlying Islands"),
    ("UY", "Uruguay"),
    ("UZ", "Uzbekistan"),
    ("VU", "Vanuatu"),
    ("VE", "Venezuela"),
    ("VN", "Viet Nam"),
    ("VG", "Virgin Islands, British"),
    ("VI", "Virgin Islands, U.S."),
    ("WF", "Wallis and Futuna"),
    ("EH", "Western Sahara"),
    ("YE", "Yemen"),
    ("ZM", "Zambia"),
    ("ZW", "Zimbabwe")
]

# Step 2: Define a function to fetch and parse data for a single country
def fetch_country_data(country_code, country_name):
    """
    Fetches and parses UN/LOCODE data for a given country.

    Parameters:
    - country_code (str): The 2-letter country code.
    - country_name (str): The name of the country.

    Returns:
    - DataFrame containing the parsed data or None if failed.
    """
    url = f"https://service.unece.org/trade/locode/{country_code.lower()}.htm"
    headers = {
        "User-Agent": "Mozilla/5.0 (compatible; UNLOCODE Scraper/1.0; +https://yourdomain.com/)"
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an exception for HTTP errors
    except requests.RequestException as e:
        print(f"Failed to fetch data for {country_code} - {country_name}: {e}")
        return None

    soup = BeautifulSoup(response.content, "html.parser")

    # Find the table containing the data
    # Based on the provided HTML, look for the table with border=1
    table = soup.find("table", {"border": "1"})
    if not table:
        print(f"No table found for {country_code} - {country_name}")
        return None

    # Extract table headers to confirm structure
    header_row = table.find("tr")
    if not header_row:
        print(f"No header row found for {country_code} - {country_name}")
        return None

    headers = [th.get_text(strip=True) for th in header_row.find_all(["th", "td"])]
    expected_headers = ["Ch", "LOCODE", "Name", "NameWoDiacritics", "SubDiv", "Function", "Status", "Date", "IATA", "Coordinates", "Remarks"]

    # Verify if the table has the expected number of columns
    if len(headers) < len(expected_headers):
        print(f"Unexpected table structure for {country_code} - {country_name}")
        return None

    # Extract table rows
    data_rows = []
    for row in table.find_all("tr")[1:]:  # Skip header row
        cells = row.find_all("td")
        if len(cells) < len(expected_headers):
            continue  # Skip incomplete rows

        # Extract text from each cell and replace non-breaking spaces
        row_data = [
            country_code,  # Country Code
            cells[0].get_text(separator=' ', strip=True).replace('\xa0', ' '),  # Ch
            cells[1].get_text(separator=' ', strip=True).replace('\xa0', ' '),  # LOCODE
            cells[2].get_text(separator=' ', strip=True).replace('\xa0', ' '),  # Name
            cells[3].get_text(separator=' ', strip=True).replace('\xa0', ' '),  # NameWoDiacritics
            cells[4].get_text(separator=' ', strip=True).replace('\xa0', ' '),  # SubDiv
            cells[5].get_text(separator=' ', strip=True).replace('\xa0', ' '),  # Function
            cells[6].get_text(separator=' ', strip=True).replace('\xa0', ' '),  # Status
            cells[7].get_text(separator=' ', strip=True).replace('\xa0', ' '),  # Date
            cells[8].get_text(separator=' ', strip=True).replace('\xa0', ' '),  # IATA
            cells[9].get_text(separator=' ', strip=True).replace('\xa0', ' '),  # Coordinates
            cells[10].get_text(separator=' ', strip=True).replace('\xa0', ' ')  # Remarks
        ]
        data_rows.append(row_data)

    if not data_rows:
        print(f"No data rows found for {country_code} - {country_name}")
        return None

    df = pd.DataFrame(data_rows, columns=["Country", "Ch", "LOCODE", "Name", "NameWoDiacritics", "SubDiv", "Function", "Status", "Date", "IATA", "Coordinates", "Remarks"])
    return df

# Step 3: Iterate through all countries and collect data
all_data = []

print("Starting data collection...")
for code, name in tqdm(country_codes, desc="Processing Countries"):
    country_df = fetch_country_data(code, name)
    if country_df is not None:
        all_data.append(country_df)
    time.sleep(1)  # Sleep to be polite to the server

# Step 4: Consolidate all data into a single DataFrame
if all_data:
    consolidated_df = pd.concat(all_data, ignore_index=True)
    # Optional: Reorder columns
    columns_order = ["Country", "Ch", "LOCODE", "Name", "NameWoDiacritics", "SubDiv", "Function", "Status", "Date", "IATA", "Coordinates", "Remarks"]
    consolidated_df = consolidated_df[columns_order]
else:
    consolidated_df = pd.DataFrame()  # Empty DataFrame
    print("No data was collected.")

# Step 5: Clean and process the DataFrame
def clean_data(df):
    """
    Cleans and processes the DataFrame.

    Parameters:
    - df (DataFrame): The DataFrame to clean.

    Returns:
    - Cleaned DataFrame.
    """
    # Replace non-breaking spaces with regular spaces and strip leading/trailing spaces
    df = df.applymap(lambda x: x.replace('\xa0', ' ').strip() if isinstance(x, str) else x)

    # Convert Date to a standard format if necessary
    # The date format in the table may vary; adjust the parsing as needed
    def parse_date(date_str):
        try:
            # Assuming the date is in YYMM or YYMMDD format
            if re.match(r"^\d{6}$", date_str):
                return pd.to_datetime(date_str, format="%y%m%d").strftime("%Y-%m-%d")
            elif re.match(r"^\d{4}$", date_str):
                return pd.to_datetime(date_str, format="%y%m").strftime("%Y-%m")
            else:
                return date_str  # Return as is if format is unrecognized
        except:
            return date_str

    df['Date'] = df['Date'].apply(parse_date)

    return df

if not consolidated_df.empty:
    consolidated_df = clean_data(consolidated_df)
    # Display the first few rows
    display(consolidated_df.head())

    # Step 6: Save the DataFrame to a CSV file with UTF-8 BOM
    csv_filename = "unlocode_consolidated.csv"
    consolidated_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"Data successfully saved to {csv_filename}")

    # Step 7: Provide a download link for the CSV file
    #files.download(csv_filename)
#else:
    #print("No data to save.")


Starting data collection...


Processing Countries: 100%|██████████| 249/249 [09:35<00:00,  2.31s/it]
<ipython-input-5-d95c0cae7c3b>:373: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('\xa0', ' ').strip() if isinstance(x, str) else x)


,Country,Ch,LOCODE,Name,NameWoDiacritics,SubDiv,Function,Status,Date,IATA,Coordinates,Remarks
0,AF,,AF ASH,Ali Shirzayi,Ali Shirzayi,URU,--3-----,RQ,2013-07,,3252N 06603E,
1,AF,,AF BAG,Bagram,Bagram,PAR,--3-----,RL,2003-07,,3457N 06915E,
2,AF,,AF BIN,Bamian,Bamian,,--34----,AI,2009-07,,,
3,AF,,AF BEP,Bazar-E-Panjwai,Bazar-E-Panjwai,KAN,--3-----,RQ,2010-01,,3133N 06527E,
4,AF,,AF BST,Bost,Bost,,---4----,AI,1999-12,,,


Data successfully saved to unlocode_consolidated.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Standardize and save final to Drive

In [ ]:
import pandas as pd

# Step 1: Load the CSV file
df = pd.read_csv('unlocode_consolidated.csv')

# Step 2: Drop columns
df = df.drop(columns=['Ch'])
df = df.drop(columns=['NameWoDiacritics'])

# Step 3: Remove spaces from the 'LOCODE' column
df['LOCODE'] = df['LOCODE'].str.replace(' ', '', regex=False)

# Step 4: Save the cleaned data to a new CSV file
df.to_csv('/content/drive/MyDrive/unlocode_final.csv', index=False)

print("CSV file saved successfully.")


CSV file saved successfully.


**Demo**

work but have problem with Excel csv



> The issue you're encountering is related to character encoding differences between how Google Sheets and Microsoft Excel interpret the CSV file. Specifically, characters like non-breaking spaces (&nbsp;) or other special characters can be misinterpreted by Excel if the encoding isn't handled correctly, leading to garbled text like AFÂ Â ASH instead of AF  ASH.

**Understanding the Problem**

1. *Encoding Mismatch:*

- Google Sheets is more forgiving and can automatically detect and handle various encodings, including UTF-8 with or without a Byte Order Mark (BOM).
- Microsoft Excel, especially older versions, often expects CSV files to be in ANSI or UTF-8 with BOM. Without the BOM, Excel might misinterpret UTF-8 encoded characters, leading to display issues like Â appearing before non-breaking spaces.

2. *Special Characters:*

Non-breaking spaces (&nbsp;) are represented as U+00A0 in Unicode. If not handled properly, they can be misread by Excel, resulting in unexpected characters.

**Solution Overview**

To resolve this issue, follow these steps:

1. *Save CSV with UTF-8 BOM:*

- Adding a BOM to the CSV file helps Excel recognize the file as UTF-8 encoded, ensuring proper rendering of special characters.

2. *Replace Non-Breaking Spaces:*

- Replace non-breaking spaces (\xa0) with regular spaces ( ) to prevent misinterpretation.

3. *Use Consistent Encoding:*

- Ensure that all text data is consistently encoded and cleaned before writing to the CSV.


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# Country codes list (you provided)
country_codes = [
    'AF', 'AX', 'AL', 'DZ', 'AS', 'AD', 'AO', 'AI', 'AQ', 'AG', 'AR', 'AM', 'AW',
    'AU', 'AT', 'AZ', 'BS', 'BH', 'BD', 'BB', 'BY', 'BE', 'BZ', 'BJ', 'BM', 'BT',
    'BO', 'BQ', 'BA', 'BW', 'BR', 'IO', 'BN', 'BG', 'BF', 'BI', 'KH', 'CM', 'CA',
    'CV', 'KY', 'CF', 'TD', 'CL', 'CN', 'CX', 'CC', 'CO', 'KM', 'CG', 'CD', 'CK',
    'CR', 'CI', 'HR', 'CU', 'CW', 'CY', 'CZ', 'DK', 'DJ', 'DM', 'DO', 'EC', 'EG',
    'SV', 'GQ', 'ER', 'EE', 'SZ', 'ET', 'FK', 'FO', 'FJ', 'FI', 'FR', 'GF', 'PF',
    'TF', 'GA', 'GM', 'GE', 'DE', 'GH', 'GI', 'GR', 'GL', 'GD', 'GP', 'GU', 'GT',
    'GG', 'GN', 'GW', 'GY', 'HT', 'HM', 'VA', 'HN', 'HK', 'HU', 'IS', 'IN', 'ID',
    'IR', 'IQ', 'IE', 'IM', 'IL', 'IT', 'JM', 'JP', 'JE', 'JO', 'KZ', 'KE', 'KI',
    'KP', 'KR', 'KW', 'KG', 'LA', 'LV', 'LB', 'LS', 'LR', 'LY', 'LI', 'LT', 'LU',
    'MO', 'MG', 'MW', 'MY', 'MV', 'ML', 'MT', 'MH', 'MQ', 'MR', 'MU', 'YT', 'MX',
    'FM', 'MD', 'MC', 'MN', 'ME', 'MS', 'MA', 'MZ', 'MM', 'NA', 'NR', 'NP', 'NL',
    'NC', 'NZ', 'NI', 'NE', 'NG', 'NU', 'NF', 'MK', 'MP', 'NO', 'OM', 'PK', 'PW',
    'PS', 'PA', 'PG', 'PY', 'PE', 'PH', 'PN', 'PL', 'PT', 'PR', 'QA', 'RE', 'RO',
    'RU', 'RW', 'BL', 'SH', 'KN', 'LC', 'MF', 'PM', 'VC', 'WS', 'SM', 'ST', 'SA',
    'SN', 'RS', 'SC', 'SL', 'SG', 'SX', 'SK', 'SI', 'SB', 'SO', 'ZA', 'GS', 'SS',
    'ES', 'LK', 'SD', 'SR', 'SJ', 'SE', 'CH', 'SY', 'TW', 'TJ', 'TZ', 'TH', 'TL',
    'TG', 'TK', 'TO', 'TT', 'TN', 'TR', 'TM', 'TC', 'TV', 'UG', 'UA', 'AE', 'GB',
    'US', 'UY', 'UZ', 'VU', 'VE', 'VN', 'VG', 'VI', 'WF', 'EH', 'YE', 'ZM', 'ZW'
]

# Base URL for the UN LOCODE page
base_url = "https://service.unece.org/trade/locode/{}.htm"

# List to store all records
data = []

# CSV headers
headers = ["Country", "Ch", "LOCODE", "Name", "NameWoDiacritics", "SubDiv", "Function", "Status", "Date", "IATA", "Coordinates", "Remarks"]

# Loop through each country code
for code in country_codes:
    url = base_url.format(code.lower())  # Make sure the country code is in lowercase in the URL
    response = requests.get(url)

    # Check if the page is accessible
    if response.status_code != 200:
        print(f"Failed to retrieve data for {code}")
        continue

    soup = BeautifulSoup(response.content, "html.parser")

    # Find the table that contains the LOCODE data
    table = soup.find('table', {"border": "1"})

    if not table:
        print(f"No table found for {code}")
        continue

    # Process rows
    rows = table.find_all('tr')[1:]  # Skip the header row
    for row in rows:
        cells = row.find_all('td')
        record = [
            code,  # Add the country code
            cells[0].text.strip(),  # Ch
            cells[1].text.strip(),  # LOCODE
            cells[2].text.strip(),  # Name
            cells[3].text.strip(),  # NameWoDiacritics
            cells[4].text.strip(),  # SubDiv
            cells[5].text.strip(),  # Function
            cells[6].text.strip(),  # Status
            cells[7].text.strip(),  # Date
            cells[8].text.strip(),  # IATA
            cells[9].text.strip(),  # Coordinates
            cells[10].text.strip()  # Remarks
        ]
        data.append(record)

# Write to CSV file
with open("un_locode_data.csv", "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerows(data)

print("Data has been written to 'un_locode_data.csv'")


Data has been written to 'un_locode_data.csv'
